In [2]:
import os
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
import os
import pandas as pd

# Directory containing the CSV files
directory = r'c:\Users\mohammed\OneDrive\Documents\QFM -S2\BankReviewIntelligence\ScrapperService\production_standalone\row_data\parcket\2024-06-08\Senegal'

# List to hold the data from each CSV file
data_frames = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".parquet"):
        file_path = os.path.join(directory, filename)
        df = pd.read_parquet(file_path)
        df['Ville'] = filename.replace('.parquet', '')
        data_frames.append(df)

# Concatenate all DataFrames
all_data = pd.concat(data_frames, ignore_index=True)

def rename_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

all_data = rename_duplicate_columns(all_data)

# Save the concatenated DataFrame to a new CSV file
all_data.to_parquet('concatenated_data_senegale.parquet', index=False)
print(all_data.shape)
all_data.head()


(277, 15)


,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Nane,Reviewer_Sart,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profil_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Ville
0,Senegal,Bignona,CBAO Bignona,Not available,"QQXF+48G, Bignona, Sénégal",Not available,Boubacar Diallo,2,NAN,il y a 5 ans,0,https://www.google.com/maps/contrib/1175237482...,NaN,NAN,Bignona
1,Senegal,Bignona,Credit Mutuel du Sénégal Bignona,+221 77 531 18 23,"RQ3C+PRP, N5, Bignona, Sénégal",Not available,Gnansou Camara,5,NAN,il y a 8 mois,0,https://www.google.com/maps/contrib/1025446772...,NaN,NAN,Bignona
2,Senegal,Bignona,Microcred Bignona,Not available,"QQWF+W78, Bignona, Sénégal",Not available,Malang bassene Bodian,5,NAN,il y a 5 ans,0,https://www.google.com/maps/contrib/1018032682...,NaN,NAN,Bignona
3,Senegal,Dakar,Banque de Dakar,+221 33 849 86 00,"13000, Sénégal",bdk.sn,Imhotep BAKHOUM,4,NAN,il y a 7 ans,0,https://www.google.com/maps/contrib/1162056137...,NaN,NAN,Dakar
4,Senegal,Dakar,LA BANQUE AGRICOLE (LBA),+221 33 839 36 36,"Place de l'indépendance, 31-33 Rue Amadou Assa...",labanqueagricole.sn,Matar Diop,3,NAN,il y a 9 ans,0,https://www.google.com/maps/contrib/1006801030...,NaN,NAN,Dakar


In [40]:
reviews = "concatenated_data.parquet"
data_set = pd.read_parquet(reviews)
data_set.rename(columns={'Reviewer_Nane': 'Name'}, inplace=True)
data_set.rename(columns={'Reviewer_Sart': 'Stars'}, inplace=True)
data_set.rename(columns={'Reviewer_Text': 'Review'}, inplace=True)
data_set.rename(columns={'Reviewer_Publish_Date': 'Date'}, inplace=True)
data_set.rename(columns={'ReviewerLike_Reaction': 'Stars'}, inplace=True)
data_set.rename(columns={'Reviewer_Like_Reaction': 'Likes'}, inplace=True)
data_set.drop(columns=['Reviewer_Profil_Link'], inplace=True)
data_set.drop(columns=['Reviewer_Owner_Reply'], inplace=True)
data_set.drop(columns=['Reviewer_Owner_Reply_Date'], inplace=True)



data_set.head()

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Name,Stars,Review,Date,Likes
0,Bénin,Abomey,Agence BOA ABOMEY,+229 21 36 51 48,"52M7+5PG, Abomey, Bénin",Not available,Daniel Alfaro,5,"Utilisez le guichet automatique, cela fonction...",il y a un an,1
1,Bénin,Abomey,UBA (United Bank of Africa) Abomey-Calavi Branch,Not available,"F954+VHF, Abomey Calavi, Bénin",Not available,Moudhil Moutairou,2,NAN,il y a 5 ans,0
2,Bénin,Abomey,Banque Atlantique Abomey-Calavi,+33 7 56 88 93 03,"C9V3+WMF, Abomey Calavi, Bénin",Not available,benedite sentibanez,3,NAN,il y a 5 ans,0
3,Bénin,Abomey,AXA Banque Agence Abomey-Calavi,Not available,"F923+6XQ, Cotonou, Bénin",Not available,Kyaam Boyz,5,NAN,il y a 2 ans,0
4,Bénin,Abomey,BOA,Not available,"F964+FFQ, RNIE2, Abomey Calavi, Bénin",Not available,Julien Lokossou,3,NAN,il y a 3 ans,0


In [41]:
import numpy as np

def fill_na_based_on_stars(row):
    if pd.isna(row['Review']):
        if row['Stars'] == 5:
            return 'Très bon'
        elif row['Stars'] == 4:
            return 'bon'
        elif row['Stars'] == 3:
            return 'neutre'
        elif row['Stars'] == 2:
            return 'mauvais'
        elif row['Stars'] == 1:
            return 'Très mauvais'
    else:
        return row['Review']
    
def map_stars_to_sentiment(stars):
    if stars >= 4:
        return 'positive'
    elif stars == 3:
        return 'neutral'
    else:
        return 'negative'

data_set['sentiment_category'] = data_set['Stars'].apply(map_stars_to_sentiment)

data_set['Review'] = data_set['Review'].astype(str).replace('NAN', np.nan)
data_set['Review'] = data_set.apply(fill_na_based_on_stars, axis=1)


In [42]:
data_set.head()

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Name,Stars,Review,Date,Likes,sentiment_category
0,Bénin,Abomey,Agence BOA ABOMEY,+229 21 36 51 48,"52M7+5PG, Abomey, Bénin",Not available,Daniel Alfaro,5,"Utilisez le guichet automatique, cela fonction...",il y a un an,1,positive
1,Bénin,Abomey,UBA (United Bank of Africa) Abomey-Calavi Branch,Not available,"F954+VHF, Abomey Calavi, Bénin",Not available,Moudhil Moutairou,2,mauvais,il y a 5 ans,0,negative
2,Bénin,Abomey,Banque Atlantique Abomey-Calavi,+33 7 56 88 93 03,"C9V3+WMF, Abomey Calavi, Bénin",Not available,benedite sentibanez,3,neutre,il y a 5 ans,0,neutral
3,Bénin,Abomey,AXA Banque Agence Abomey-Calavi,Not available,"F923+6XQ, Cotonou, Bénin",Not available,Kyaam Boyz,5,Très bon,il y a 2 ans,0,positive
4,Bénin,Abomey,BOA,Not available,"F964+FFQ, RNIE2, Abomey Calavi, Bénin",Not available,Julien Lokossou,3,neutre,il y a 3 ans,0,neutral


In [43]:
from nltk.stem.isri import ISRIStemmer
Text = data_set['Review']
bank = data_set['Bank_Name']
Sentiment  = data_set['sentiment_category']

preprossed_text = []
stop_words = set(stopwords.words('french'))
stop_words.add('Je')
stop_words.add('vais')
stop_words.add('it')
st = ISRIStemmer()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.split()
    words = [st.stem(word) for word in text if word not in stop_words]
    return ' '.join(words)

for text in Text:
    text = preprocess_text(text)
    preprossed_text.append(text)


data = pd.DataFrame({'Bank':bank,'preprocessed_review':preprossed_text , 'Sentiment':Sentiment})
data.head()

,Bank,preprocessed_review,Sentiment
0,Agence BOA ABOMEY,utilisez guichet automatique cela fonctionne p...,positive
1,UBA (United Bank of Africa) Abomey-Calavi Branch,mauvais,negative
2,Banque Atlantique Abomey-Calavi,neutre,neutral
3,AXA Banque Agence Abomey-Calavi,très bon,positive
4,BOA,neutre,neutral


In [44]:

"""model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)"""

'model_name = "nlptown/bert-base-multilingual-uncased-sentiment"\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name)\nclassifier = pipeline(\'sentiment-analysis\', model=model, tokenizer=tokenizer)'

In [45]:
"""def classify_sentiment(review):
    result = classifier(review)[0]
    return result['label']


data['sentiment'] = data['preprocessed_review'].apply(classify_sentiment)
data.head()"""

"def classify_sentiment(review):\n    result = classifier(review)[0]\n    return result['label']\n\n\ndata['sentiment'] = data['preprocessed_review'].apply(classify_sentiment)\ndata.head()"

In [77]:
topics = {
    "positive": {
        "bon": ["généralement bon"],
        "service_client": ["excellent service client", "réponses rapides aux demandes", "personnel serviable"],
        "produits_financiers": ["produits de haute qualité", "conditions avantageuses", "innovations régulières"],
        "expérience_utilisateur": ["interface intuitive", "navigation fluide", "facilité d'utilisation des services en ligne"],
        "gestion_des_comptes": ["gestion transparente des comptes", "outils efficaces de suivi financier", "faibles frais bancaires"],
        "sécurité": ["sécurité des transactions garantie", "protection renforcée des données", "gestion proactive des risques"],
        "localisation_accessibilité": ["emplacements pratiques", "accessibilité optimale aux agences", "guichets automatiques disponibles"],
        "services_additionnels": ["offres spéciales attrayantes", "programmes de fidélité avantageux", "services d'investissement performants"]
    },
    "neutral": {
        "neutre": ["généralement neutre"],
        "service client": ["service client standard", "réponses acceptables", "temps d'attente modéré"],
        "produits financiers": ["produits standards", "conditions habituelles", "aucune innovation notable"],
        "expérience utilisateur": ["interface correcte", "navigation fonctionnelle", "services en ligne satisfaisants"],
        "gestion des comptes": ["gestion des comptes moyenne", "informations claires mais basiques", "frais bancaires standard"],
        "sécurité": ["mesures de sécurité standards", "réponses adéquates aux risques", "sécurité des données respectée"],
        "localisation accessibilité": ["localisation acceptable des agences", "accessibilité correcte", "services de guichets automatiques suffisants"],
        "services additionnels": ["offres de crédit ordinaires", "programmes de fidélité standards", "services d'investissement réguliers"]
    },
    "negative": {
        
        "service client": ["service client médiocre", "réponses lentes aux demandes", "manque d'assistance"],
        "produits financiers": ["produits de qualité inférieure", "conditions défavorables", "manque d'adaptabilité"],
        "interface en ligne et application ": ["interface confuse", "navigation difficile", "problèmes fréquents avec les services en ligne"],
        "gestion des comptes": ["difficulté dans la gestion des comptes", "informations confuses ou trompeuses", "frais bancaires élevés"],
        "sécurité": ["préoccupations sérieuses de sécurité", "incidents de fraude fréquents", "protection des données insuffisante"],
        "localisation accessibilité": ["mauvaise localisation des agences", "accessibilité limitée", "manque de guichets automatiques"],
        "services additionnels": ["offres de crédit peu avantageuses", "programmes de fidélité décevants", "services d'investissement sous-performants"]
    }
}




In [55]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Load your data
reviews = data['preprocessed_review'].tolist()

# Initialize BERTopic model
topic_model = BERTopic(language="multilingual")

# Fit the model on your reviews
topics, probs = topic_model.fit_transform(reviews)

# Get topic information
topic_info = topic_model.get_topic_info()

# Display topics
print(topic_info)

No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.


    Topic  Count                                  Name  \
0      -1     18    -1_ny_sasseoir_publiques_toilettes   
1       0    697         0_très_bon_bonnes_prestations   
2       1    255           1_bon_bien_daccord_déjeuner   
3       2    221                           2_neutre___   
4       3    176                      3_mauvais_très__   
5       4     81                          4_mauvais___   
6       5     80     5_banque_bancaire_transfert_maroc   
7       6     56          6_endroit_haussa_bel_épaules   
8       7     46      7_service_meilleur_client_puisse   
9       8     23                      8_mauvais_très__   
10      9     16    9_excellent_super_cool_merveilleux   
11     10     14      10_voir_comportement_lent_agence   
12     11     14  11_accueil_personnel_accueillant_jai   
13     12     13                             12_bon___   

                                       Representation  \
0   [ny, sasseoir, publiques, toilettes, sièges, p...   
1   [très, bon,

In [62]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [65]:
sentences = [
    "Votre service client est exceptionnel et les produits financiers sont innovants.",
    "Les frais bancaires sont trop élevés."
]

# Tokenize sentences
inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Get BERT embeddings
with torch.no_grad():
    outputs = model(**inputs)

# The embeddings are in the 'last_hidden_state'
embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

# To get sentence-level embeddings, we can mean-pool the token embeddings
sentence_embeddings = torch.mean(embeddings, dim=1)  # Shape: (batch_size, hidden_size)

from sklearn.cluster import KMeans

# Number of topics
num_topics = 2

# Fit KMeans
kmeans = KMeans(n_clusters=num_topics, random_state=42)
kmeans.fit(sentence_embeddings)

# Assign topics to sentences
topics = kmeans.labels_

# Print the topics for each sentence
for sentence, topic in zip(sentences, topics):
    print(f"Sentence: {sentence}")
    print(f"Assigned Topic: {topic}\n")



Sentence: Votre service client est exceptionnel et les produits financiers sont innovants.
Assigned Topic: 0

Sentence: Les frais bancaires sont trop élevés.
Assigned Topic: 1



In [81]:
from sklearn.preprocessing import Normalizer
    
vectrizer = TfidfVectorizer()

X = vectrizer.fit_transform(data['preprocessed_review'])
X.shape

model = TruncatedSVD(n_components=200, n_iter=200, random_state=42)

X = model.fit_transform(X)
normalizer = Normalizer(copy=False)
lsa_matrix = normalizer.fit_transform(X)


# Deviding data into posetive, negative and neutral
positive_data = data[data['Sentiment'] == 'positive']
neutral_data = data[data['Sentiment'] == 'neutral']
negative_data = data[data['Sentiment'] == 'negative']


def get_topic_similarity(topic, review):
    review_vector = vectrizer.transform([review])
    review_vector = model.transform(review_vector)
    topic_vector = vectrizer.transform([topic])
    topic_vector = model.transform(topic_vector)
    similarity = cosine_similarity(review_vector, topic_vector)
    return similarity[0][0]

def get_review_topics(review, sentiment):
    main_topic_similarities = {}
    for main_topic in topics[sentiment].keys():
        similarity = get_topic_similarity(main_topic, review)
        main_topic_similarities[main_topic] = similarity
    print(main_topic_similarities)   
    
    # Choisir le sujet principal avec la plus grande similarité
    chosen_main_topic = max(main_topic_similarities, key=main_topic_similarities.get)
    
    # Calculer la similarité avec les sous-sujets du sujet principal choisi
    sub_topic_similarities = {}
    for sub_topic in topics[sentiment][chosen_main_topic]:
        similarity = get_topic_similarity(sub_topic, review)
        sub_topic_similarities[sub_topic] = similarity
    
    # Choisir le sous-sujet avec la plus grande similarité
    chosen_sub_topic = max(sub_topic_similarities, key=sub_topic_similarities.get)
    
    return chosen_main_topic, chosen_sub_topic




# Exemple d'utilisation
review = "L'application pour acceder a mon compte bug. On n'arrive pas a se connecter facilement."
sentiment = 'negative'
main_topic, sub_topic = get_review_topics(review, sentiment)
print(f"Review: {review}")
print(f"Main Topic: {main_topic}")
print(f"Sub Topic: {sub_topic}")    

{'service client': 0.04128671294180376, 'produits financiers': 0.0014901135693074841, 'interface en ligne et application ': 0.5972428598016069, 'gestion des comptes': -0.013883131942973869, 'sécurité': -0.002340188968845761, 'localisation accessibilité': 0.0, 'services additionnels': -0.0021368853886658563}
Review: L'application pour acceder a mon compte bug. On n'arrive pas a se connecter facilement.
Main Topic: interface en ligne et application 
Sub Topic: problèmes fréquents avec les services en ligne


In [48]:
def get_topics_from_reviews(reviews, sentiment):
    main_topics = []
    sub_topics = []
    for review in reviews:
        main_topic, sub_topic = get_review_topics(review, sentiment)
        main_topics.append(main_topic)
        sub_topics.append(sub_topic)
    return main_topics, sub_topics

positive_reviews = positive_data['preprocessed_review']
neutral_reviews = neutral_data['preprocessed_review']
negative_reviews = negative_data['preprocessed_review']

positive_main_topics, positive_sub_topics = get_topics_from_reviews(positive_reviews, 'positive')
neutral_main_topics, neutral_sub_topics = get_topics_from_reviews(neutral_reviews, 'neutral')
negative_main_topics, negative_sub_topics = get_topics_from_reviews(negative_reviews, 'negative')

positive_data['main_topic'] = positive_main_topics
positive_data['sub_topic'] = positive_sub_topics

neutral_data['main_topic'] = neutral_main_topics
neutral_data['sub_topic'] = neutral_sub_topics

negative_data['main_topic'] = negative_main_topics
negative_data['sub_topic'] = negative_sub_topics


C:\Users\mohammed\AppData\Local\Temp\ipykernel_23284\4073852484.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_data['main_topic'] = positive_main_topics
C:\Users\mohammed\AppData\Local\Temp\ipykernel_23284\4073852484.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_data['sub_topic'] = positive_sub_topics
C:\Users\mohammed\AppData\Local\Temp\ipykernel_23284\4073852484.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [49]:
Final_result = pd.concat([positive_data, neutral_data, negative_data], ignore_index=True)
Final_result.to_csv('Final_result.csv', index=False)
Final_result.head()


,Bank,preprocessed_review,Sentiment,main_topic,sub_topic
0,Agence BOA ABOMEY,utilisez guichet automatique cela fonctionne p...,positive,sécurité,gestion proactive des risques
1,AXA Banque Agence Abomey-Calavi,très bon,positive,bon,généralement bon
2,BSIC BENIN SA - kpota,très bon,positive,bon,généralement bon
3,CLCAM ABOMEY-CALAVI AGENCE SIEGE,bon,positive,bon,généralement bon
4,Nsia Banque Bénin,très bon,positive,bon,généralement bon


In [50]:
import torch
import torch.nn as nn

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.relu =  nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 3)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_ids, attention_mask):
        _, output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.fc1(output)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc5(x)
        return self.softmax(x)





In [51]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Example data
texts = ["The customer service at this bank is excellent.",
         "I had a terrible experience with the bank's online banking system.",
         "The loan process was quick and easy.",
         "The bank charges too many fees.",
         "I love the mobile app provided by the bank."]
labels = [1, 0, 1, 0, 1]  # Example binary labels (positive=1, negative=0)

# Split data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Create datasets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            torch.tensor(val_labels))

# Load pretrained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 5
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluate on validation set
model.eval()
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
predictions = []
true_labels = []
for batch in val_loader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=1).tolist())
    true_labels.extend(batch[2].tolist())

# Calculate metrics
print(classification_report(true_labels, predictions))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\moham

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

